In [ ]:
import numpy as np
import pandas as pd
import csv
import os
import sys
import sklearn
import tensorflow as tf
from tensorflow import keras
import math

import gc

#modeling
from keras import backend as K
from keras.models import Model
from keras.layers import *
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
#import transformers

#image preprocessing
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from PIL import Image
import cv2

#visualization
import matplotlib.pyplot as plt
#import seaborn as sns

#warnings
import warnings
warnings.filterwarnings('ignore')


import cudf, cuml, cupy
from cuml.neighbors import NearestNeighbors


# RESTRICT TENSORFLOW TO 1GB OF GPU RAM
# SO THAT WE HAVE 15GB RAM FOR RAPIDS
LIMIT = 14
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*LIMIT)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    #print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)


K.clear_session()
np.random.seed(2021); tf.random.set_seed(2021)

In [ ]:
train = pd.read_csv('../input/shopee-product-matching/train.csv')
#N_tot = len(train)
#N_index = np.arange(N_tot)
#train['index'] = N_index

train_jpg_directory = '../input/shopee-product-matching/train_images'

train_image_path = []
for img in train.image:
    train_image_path.append(os.path.join(train_jpg_directory, img))
train['img_path'] = train_image_path

In [ ]:
from sklearn.model_selection import GroupKFold
skf = GroupKFold(5)
train['fold'] = -1
for i, (train_idx, valid_idx) in enumerate(skf.split(X=train, groups=train['label_group'])):
    train.loc[valid_idx, 'fold'] = i

In [ ]:
training_set = []
validation_set = []

for i in range(5):
    train_fold = train[train['fold']!=i].reset_index(drop=True)
    label_dict = train_fold.groupby('label_group').posting_id.agg('unique').to_dict()
    train_fold['target'] = train_fold.label_group.map(label_dict)
    #train_fold['indexing'] = np.arange(len(train_fold))
    
    valid_fold = train[train['fold']==i].reset_index(drop=True)
    label_dict = valid_fold.groupby('label_group').posting_id.agg('unique').to_dict()
    valid_fold['target'] = valid_fold.label_group.map(label_dict)
    
    training_set.append(train_fold)
    validation_set.append(valid_fold)

In [ ]:
IMAGE_SIZE = [256, 256]

In [ ]:
def create_batch(train, batch_index, label_dist):
    x_inputs = np.zeros((len(batch_index), 256, 256, 3))
    x_sim = np.ones((len(batch_index), len(batch_index))) * -1
    for i, iindex in enumerate(batch_index):
        x_anchor = train.iloc[iindex].img_path
        x_anchor = tf.io.read_file(x_anchor)
        x_anchor = tf.image.decode_jpeg(x_anchor, channels=3)
        x_anchor = tf.image.resize(x_anchor, IMAGE_SIZE)

        x_inputs[i] = x_anchor
        for k, rindex in enumerate(batch_index):
            #print(train.iloc[iindex].indexing)
            if rindex in label_dist[train.iloc[iindex].indexing]:
                x_sim[i, k] = 1.0
    
    return x_inputs, x_sim

In [ ]:
from tensorflow.keras.applications import EfficientNetB0

In [ ]:
def build_model():
    model = EfficientNetB0(include_top=False, weights='../input/effnetb0/efficientnetb0_notop.h5', pooling='avg', input_shape=None)
    x = Dropout(rate = 0.2, name = "added_dropout")(model.output)
    x = Dense(units = 128, name = "added_fc_layer")(x)
    x = BatchNormalization(name = "added_bn_layer")(x)
    x = Lambda(lambda x: K.l2_normalize(x,axis=1))(x)
    model = Model(inputs=model.input, outputs=x)
    return model

In [ ]:
def get_batch(train, batch_index):
    x_inputs = np.zeros((len(batch_index), 256, 256, 3))
    for i, iindex in enumerate(batch_index):
        x_anchor = train.iloc[iindex].img_path
        x_anchor = tf.io.read_file(x_anchor)
        x_anchor = tf.image.decode_jpeg(x_anchor, channels=3)
        x_anchor = tf.image.resize(x_anchor, IMAGE_SIZE)

        x_inputs[i] = x_anchor
    return x_inputs

In [ ]:
def pair_loss(x_output, x_cur, similarity):
    
    prod = tf.matmul(x_output, x_cur.T)
    
    loss = tf.maximum(0, 0.5 - tf.multiply(prod, similarity))
    
    return tf.reduce_mean(loss)

In [ ]:
valid_feats = []

for i in range(5):
    train = training_set[i]
    valid = validation_set[i]
    
    train['indexing'] = np.arange(len(train))
    label_dict = train.groupby('label_group').indexing.agg('unique').to_dict()
    train['target'] = train.label_group.map(label_dict)
    
    N = len(train)
    print(N)
    
    label_list = []
    similar_img = {} 
    for il in range(N):
        similar_img[il] = train.iloc[il].target.tolist()    
    
    for k, v in label_dict.items():
        label_list.append(v.tolist())
        
    N = len(label_list)
    
    N_index = np.arange(N)
    
    batchsize = 15
    
    ITER = N // batchsize
    if N%batchsize != 0:
        ITER += 1
    
    K.clear_session()
    op = keras.optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False)
    model = build_model()
    
    loss_fn = pair_loss

    for ip in range(8):
        np.random.shuffle(N_index)
        ITER = 0
        out_it = 0
        
        while ITER<N:
            curlen = 0
            batch_index = []
            while ITER < N:
                QQQ = len(label_list[N_index[ITER]])
                if curlen + QQQ < 80:
                    batch_index.extend(label_list[N_index[ITER]])  
                    curlen += QQQ
                    ITER += 1
                else:
                    break
            if len(batch_index) == 0:
                print("batch_index ==== 0!!! wrong")
                exit()
            x_input, sim = create_batch(train, batch_index, similar_img)
            #print(x_input.shape)
            x_cur = model.predict(x_input)
            #print(x_cur.shape)

            with tf.GradientTape() as tape:
                x_output = model([x_input], training=True)
                loss_value = loss_fn(x_output, x_cur, sim)
            grads = tape.gradient(loss_value, model.trainable_weights)
            op.apply_gradients(zip(grads, model.trainable_weights))
            if ITER // 1000 == out_it:
                print("In fold %d, epoch %d, iteration %d, loss value = %.4f"%(i, ip, ITER, loss_value) )
                out_it += 1

    N = len(valid)
    N_index = np.arange(N)
    batchsize = 128
    ITER = N // batchsize
    if N%batchsize != 0:
        ITER += 1
    valid_feat = np.zeros((N, 128))
    print(N)
    for it in range(ITER):
        train_batch = N_index[it*batchsize: min((it + 1)*batchsize, N)]
        x_batch = get_batch(valid, train_batch)
        x_cur = model.predict(x_batch)
        valid_feat[train_batch] = x_cur
    
    valid_feats.append(valid_feat)

In [ ]:
thresholdlist = list(np.linspace(0.3,0.8,15))
print(thresholdlist)

acc_threshold = np.zeros((5, len(thresholdlist)))

In [ ]:
def computeFscore(true, pred):
    n = len( np.intersect1d(true, pred) )
    return 2*n / (len(true)+len(pred))

In [ ]:
batch_size = 1024

for thi, thval in enumerate(thresholdlist):
    print("begining")
    scores = []
    score = 0
    for i in range(5):
        valid = validation_set[i]
        valid_feat = valid_feats[i]
        N_size = valid_feat.shape[0]
        valid_feat = cupy.asarray(valid_feat)
             
        N_index = np.arange(N_size)
        num_part = N_size//batch_size
        if N_size%batch_size !=0:
            num_part += 1

        f1_score = 0.0
        
        for k in range(num_part):
            batch_index = N_index[i*batch_size: min((i+1)*batch_size, len(N_index))]
            batch_feat = valid_feat[batch_index]
            distances = cupy.matmul(batch_feat, valid_feat.T)
            for s in range(len(batch_index)):
                IDX = cupy.where(distances[s] >= thval)[0]
                #print(IDX)
                pred = valid.iloc[cupy.asnumpy(IDX)].posting_id.values
                f1_score += computeFscore(valid.iloc[batch_index[s]].target, pred)
        f1_score = f1_score/N_size
        
        acc_threshold[i, thi] = f1_score
        print(f1_score)
print(acc_threshold)
print(np.mean(acc_threshold, axis = 0))